# RAG System for Survey Variable Resolution

This Jupyter Notebook is designed to implement a RAG (Retrieval-Augmented Generation) system. The primary purpose of this system is to resolve user-submitted survey variable data and find other variables that are likely to match.

## Overview

The RAG system leverages the power of retrieval-based and generative models for machine learning. It uses a two-step process:

1. **Retrieval**: The system retrieves relevant documents (in this case, survey variables) from a Pinecone DB knowledge source based on the user-submitted data.

2. **Generation**: The system then uses the retrieved documents to generate a response using a Cohere Reranking LLM and OpenAI's ChatOpenAI GPT 3.5 model. This response includes other variables that have a high likelihood of matching the user-submitted data.

## Usage

To use this notebook, input your survey variable data using the `input_data` directory. The RAG system will process this data, retrieve relevant variables from the knowledge source, and generate a list of variables that are likely to match your input.

## Benefits

The RAG system provides a powerful tool for survey data analysis. It can help identify patterns and correlations in the data, which can be invaluable for research and decision-making.

Please note that the accuracy of the system's output depends on the quality and comprehensiveness of the knowledge source. Therefore, it's crucial to continually update and expand the knowledge source to improve the system's performance.

## Setup

In [14]:
import os
import json
import pandas as pd
from langchain.docstore.document import Document
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers.multi_query import MultiQueryRetriever

import cohere
from pinecone import Pinecone
from pinecone import ServerlessSpec



from getpass import getpass
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings


In [42]:
import os
from getpass import getpass
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

#### API KEYS

In [44]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') or getpass("OpenAI API Key: ")
COHERE_API_KEY = os.getenv('COHERE_API_KEY') or getpass("COHERE API Key: ")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY") or getpass("Enter your Pinecone API key: ")



#### Configuration Values

In [15]:
PINECONE_INDEX = "biolincc-labels-001" # Name of the Pinecone index
TEMPERATURE = 0
VECTOR_TEXT_FIELD = "text"
EMBEDDING_MODEL = "text-embedding-ada-002"


#### Logging

In [12]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

### Embedding and Vector DB Setup

In [6]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
spec = ServerlessSpec(
    cloud="aws", region="us-west-2"
)

In [16]:
import time

index_name = PINECONE_INDEX
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
  raise Exception("Pinecone index does not exist")

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 388951}},
 'total_vector_count': 388951}

In [17]:
embed = OpenAIEmbeddings(
    model=EMBEDDING_MODEL, openai_api_key=OPENAI_API_KEY, disallowed_special=()
)

In [19]:
from langchain.vectorstores import Pinecone

vectorstore = Pinecone(index, embed.embed_query, VECTOR_TEXT_FIELD)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


## Parse User Input

In [3]:
# Specify the path to your CSV file
csv_file_path = 'input_data/test_data_dictionary.csv'

# Read the CSV file and convert it into a DataFrame
df = pd.read_csv(csv_file_path)

In [4]:
df.describe()

,Sample/Subject Count
count,336.000000
mean,7418.211310
std,3661.967493
min,12.000000
25%,4388.000000
50%,10122.000000
75%,10370.000000
max,12240.000000


In [5]:
df = df.head(5)
# The column we need to input into the RAG is 'description'
# While in test mode this is all we want to do

In [21]:
user_variables = df['description']

## Setup Retriever

We'll try this with two prompts, both encourage more variety in search queries.

**Prompt A**
```
Your task is to generate 3 different search queries that aim to
answer the user question from multiple perspectives.
Each query MUST tackle the question from a different viewpoint,
we want to get a variety of RELEVANT search results.
Provide these alternative questions separated by newlines.
Original question: {question}
```


**Prompt B**
```
Your task is to compare the results a variable lookup and rerankings
with the query Data Dictionary. The user would like to know how
closely each of their varaiables aligns with varaibles in the data corpus.
In order to do this, we need to compare the results of the original
query for each variable with the results from the variable lookup.
If the variable has a high similarity with the results from the variable
lookup, then we should say it is a good match. With variables that have no match,
we should say that there is no match. If the variable has a low similarity with the
results from the variable lookup, then we should discuss the matches
that result and how the user may address this variable for better alignment.
We want to provide the user with as much information as we can on how to
align their variables with the data corpus.
Original question: {question}
```

In [7]:
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [9]:
# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)

output_parser = LineListOutputParser()


### Prompt Template

In [69]:

template = """
    You are a data curator whos role is to harmonize biological variables in the NHLBI (National Heart Lung and Blood) data repository. 
    You are tasked with evaluating input variables from data dictionary that describes new data that will be added to the existing pool of variables in the repository.
    For each new variable, a vector search engine has returned the three nearest existing variables found in the data repository. 
    Your job is determine which of these existing variables, if any, is the best fit for harmonizing the new variable to the existing variable given your understanding of the underlying scientific principles.
    You must rationalize why the selected existing variable was chosen over the others, provide context to the relevancy of each existing variable to the new variable.
    Then you are to provide the user with as much information as we can on how they can align their new variable with the selected existing variable.
    
    When there is no obvious match provide additional context for why you can't make a determination.
    
    Contexts:
    {contexts}

    Original question: {query}
"""

GEN_PROMPT = PromptTemplate(
    # input_variables=["question"],
    input_variables=["query", "contexts"],
    template=template,
)
llm = ChatOpenAI(temperature=TEMPERATURE, openai_api_key=OPENAI_API_KEY)

# Chain
gen_chain = LLMChain(llm=llm, prompt=GEN_PROMPT, output_parser=output_parser)

In [63]:
### IS THIS NEEDED? TBD
# query_template = """
#     You are a data curator whos role is to harmonize biological variables in the NHLBI (National Heart Lung and Blood) data repository. 
#     You are tasked with evaluating input variables from data dictionary that describes new data that will be added to the existing pool of variables in the repository.
    
#     Original query: {query}
# """

# QUERY_PROMPT = PromptTemplate(
#     # input_variables=["question"],
#     input_variables=["query"],
#     template=query_template,
# )
# llm = ChatOpenAI(temperature=TEMPERATURE, openai_api_key=OPENAI_API_KEY)

# # Chain
# llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

## Rerank QA

In [70]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_community.llms import Cohere

In [71]:
cohere_rerank = CohereRerank(cohere_api_key=COHERE_API_KEY)
retriever=ContextualCompressionRetriever(
    base_compressor=cohere_rerank, 
    base_retriever=vectorstore.as_retriever() 
)


In [65]:
# retriever = MultiQueryRetriever(
#     # retriever=vectorstore.as_retriever(), llm_chain=llm_chain, parser_key="lines"
#     retriever=ContextualCompressionRetriever(
#     base_compressor=cohere_rerank, 
#     base_retriever=vectorstore.as_retriever() 
#     ),
#     llm_chain=llm_chain,
#     parser_key="lines"
# )  # "lines" is the key (attribute name) of the parsed output

## Connecting RAG elements

In [72]:
from langchain.chains import TransformChain
from langchain.chains import SequentialChain

In [73]:
def retrieval_transform(inputs: dict) -> dict:
    docs = retriever.get_relevant_documents(query=inputs["question"])
    docs = [d.page_content for d in docs]
    docs_dict = {
        "query": inputs["question"],
        "contexts": "\n---\n".join(docs)
    }
    return docs_dict

retrieval_chain = TransformChain(
    input_variables=["question"],
    output_variables=["query", "contexts"],
    transform=retrieval_transform
)

In [74]:

rag_chain = SequentialChain(
    chains=[retrieval_chain, gen_chain],
    input_variables=["question"],  # we need to name differently to output "query"
    output_variables=["query", "contexts", "text"]
)

### Execute the Model

In [33]:
variable_label_1 = "Highest level of formal/academic education achieved"
variable_label_2 = "lesion MLD and %DS in the persistent region"
variable_label_3 = "Doing things that make you feel valued"
variable_label_4 = "Acute Exacerbation, Medical Monitor Identified Even"
variable_label_5 = "COVID-19 specimen #1 type (e.g. nasopharyngeal swab)"

In [75]:
out = rag_chain({"question": variable_label_1})


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [76]:
out["text"]

LineList(lines=['Based on the provided context, the new variable "Highest level of formal/academic education achieved" can be harmonized with the existing variable "Education level" in the data repository. ', '', 'The existing variable "Education level" is the best fit for harmonizing the new variable because it captures the same concept of the highest level of education attained by an individual. The terms "formal" and "academic" in the new variable are encompassed within the broader category of "Education level", making it a suitable match.', '', 'The other two existing variables returned by the vector search engine may not be as relevant. "Highest education level achieved" is very similar to the new variable but lacks the specificity of including both formal and academic education. "Highest level of education" is also related but may not capture the distinction between formal and informal education.', '', 'To align the new variable with the selected existing variable "Education leve

In [77]:
docs = retriever.get_relevant_documents(
    query=variable_label_1
)


In [78]:
docs

[Document(page_content='Highest level of formal/academic education achieved', metadata={'Study': 'OMNI', 'Variable': 'Q1_EDUCG', 'id': 360430.0, 'relevance_score': 0.9984988}),
 Document(page_content='Highest education level achieved', metadata={'Study': 'FHSCOH', 'Variable': 'KAEDUC', 'id': 155426.0, 'relevance_score': 0.94938856}),
 Document(page_content='Highest level of education', metadata={'Study': 'GHCCHN', 'Variable': 'DI2', 'id': 201564.0, 'relevance_score': 0.728362})]

In [ ]:
docs = [d.page_content for d in docs]